In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, load_from_disk
from transformers import DataCollatorForLanguageModeling
import transformers

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="openai-community/gpt2",
    use_fast=True,
    trust_remote_code=True, 
    cache_dir="./cache"
)

def tokenize(examples):
    return tokenizer(examples["prompt"])

len(tokenizer)
#dataset = load_dataset("fka/awesome-chatgpt-prompts", cache_dir="./cache", trust_remote_code=True)

c:\Users\mingx\anaconda3\envs\mingxuan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\mingx\anaconda3\envs\mingxuan\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


50257

In [3]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="openai-community/gpt2",
    trust_remote_code=True,
    cache_dir="./cache",
    attn_implementation="sdpa",
    torch_dtype=torch.float16,
    device_map="auto"
)

In [37]:
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=1e-4, 
    betas=[0.9, 0.95], 
    eps=1e-8,
    weight_decay=0.0,
    fused=True
)

In [4]:
model.state_dict().keys()


odict_keys(['transformer.wte.weight', 'transformer.wpe.weight', 'transformer.h.0.ln_1.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.attn.c_attn.weight', 'transformer.h.0.attn.c_attn.bias', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.0.ln_2.weight', 'transformer.h.0.ln_2.bias', 'transformer.h.0.mlp.c_fc.weight', 'transformer.h.0.mlp.c_fc.bias', 'transformer.h.0.mlp.c_proj.weight', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.1.ln_1.weight', 'transformer.h.1.ln_1.bias', 'transformer.h.1.attn.c_attn.weight', 'transformer.h.1.attn.c_attn.bias', 'transformer.h.1.attn.c_proj.weight', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.ln_2.weight', 'transformer.h.1.ln_2.bias', 'transformer.h.1.mlp.c_fc.weight', 'transformer.h.1.mlp.c_fc.bias', 'transformer.h.1.mlp.c_proj.weight', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.2.ln_1.weight', 'transformer.h.2.ln_1.bias', 'transformer.h.2.attn.c_attn.weight', 'transformer.h.2.attn.

In [5]:
import re
def _compile_target_modules(target_modules):
    if target_modules is None:
        return None
    if isinstance(target_modules, str):
        target_modules = [target_modules]
    
    compiled_patterns = []
    for pattern in target_modules:
        if pattern.startswith("re:"):
            # If the pattern starts with 're:', compile it as a regular expression
            compiled_patterns.append(re.compile(pattern[3:], re.IGNORECASE))
        else:
            # Compile it to match the pattern anywhere in the string
            escaped_pattern = re.escape(pattern).replace(r'\*', '.*')
            compiled_patterns.append(re.compile(f".*{escaped_pattern}.*", re.IGNORECASE))
    
    return compiled_patterns

In [9]:
target_modules = ["c_attn.weight", "c_proj.weight", "c_fc.weight"]
compiled_patterns = _compile_target_modules(target_modules)

In [11]:
total_target_params = 0
for name, param in model.named_parameters():
    if any(pattern.search(name) for pattern in compiled_patterns):
        print(name)
        total_target_params += param.numel()
print(f"Total number of candidate parameters for pruning: {total_target_params:,}")

transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_proj.weight
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_proj.weight
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_proj.weight
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_proj.weight
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_proj.weight
transformer.h.2.mlp.c_fc.weight
transformer.h.2.mlp.c_proj.weight
transformer.h.3.attn.c_attn.weight
transformer.h.3.attn.c_proj.weight
transformer.h.3.mlp.c_fc.weight
transformer.h.3.mlp.c_proj.weight
transformer.h.4.attn.c_attn.weight
transformer.h.4.attn.c_proj.weight
transformer.h.4.mlp.c_fc.weight
transformer.h.4.mlp.c_proj.weight
transformer.h.5.attn.c_attn.weight
transformer.h.5.attn.c_proj.weight
transformer.h.5.mlp.c_fc.weight
transformer.h.5.mlp.c_proj.weight
transformer.h.6.attn.c_attn.weight
transformer.h.6.attn.c_proj.weight
transformer.h.6.mlp.c_fc.weight
transformer.h.6.mlp.c_proj.weight
transformer.h.7.attn.c_attn.weight
transformer.h